In [21]:
import numpy as np
from numpy import linalg
from sympy import *

In [22]:
# n = 4
n = 2
m = 10
# m = 20
# m = 11 for kowalik-osborne function

In [23]:
# x = np.array([symbols('x{0}'.format(i + 1)) for i in range(n)])
# u = np.array([symbols('u{0}'.format(i + 1)) for i in range(m)])
# y = np.array([symbols('y{0}'.format(i + 1)) for i in range(m)])

In [24]:
# def create_kowalik_osborne_function(x, u, y):
#     return np.array([simplify('{y_i} - ({x1}*({u_i}*{u_i} + {u_i}*{x2})) / ({u_i}*{u_i} + {u_i}*{x3} + {x4})'.format())])

In [25]:
# Brown - Dennis function
def create_brown_denis_function(x, m):
    arr = []
    for i in range(m):
        t = symbols('t{0}'.format(i + 1))
        t_i = t.subs({t: (i + 1) / 5})
        arr.append(simplify('({x_1} + {t_i}*{x_2} - exp({t_i}))^2 + ({x_3} + {x_4}*sin({t_i}) - cos({t_i}))^2'
        .format(x_1 = x[0], x_2 = x[1], x_3 = x[2], x_4 = x[3], t_i = t_i)))
    
    return np.array(arr)

In [26]:
# Jenrich - Sampson function
def CreateJenrichSampsonFunction(x, m):
    arr = []
    for i in range(m):
        # t = symbols('t{0}'.format(i + 1))
        # t_i = t.subs({t: (i + 1) / 5})
        arr.append(simplify('2 + 2*{i} - exp({i}*{x_1}) - exp({i}*{x_2})'.format(x_1 = x[0], x_2 = x[1], i = i + 1)))
    
    return np.array(arr)

In [27]:
def createFunctions(x, u, y, m):
    arr = []
    for i in range(m):
        t_i = (u[i] - 425) / 195
        arr.append(simplify('{x_1}*exp({t_i}*{x_3}) + {x_2}*exp({t_i}*{x_4}) - {y_i}'
            .format(x_1= x[0], x_2 = x[1], x_3 = x[2], x_4 = x[3], t_i = t_i, y_i = y[i])))
    
    return np.array(arr)

In [29]:
x = np.array([symbols('x{0}'.format(i + 1)) for i in range(4)])
u = np.array([230, 295, 360, 425, 490, 555, 620])
y = np.array([64.0, 66.0, 69.5, 74.0, 80.8, 91.0, 103.5], dtype = 'float')
F = createFunctions(x, u, y, 7)
print(F)
x0 = np.array([25, 45, 1, 0])

[1.0*x1*exp(-1.0*x3) + 1.0*x2*exp(-1.0*x4) - 64.0
 1.0*x1*exp(-0.6666666666666666*x3) + 1.0*x2*exp(-0.6666666666666666*x4) - 66.0
 1.0*x1*exp(-0.3333333333333333*x3) + 1.0*x2*exp(-0.3333333333333333*x4) - 69.5
 x1 + x2 - 74.0
 x1*exp(0.3333333333333333*x3) + x2*exp(0.3333333333333333*x4) - 80.8
 x1*exp(0.6666666666666666*x3) + x2*exp(0.6666666666666666*x4) - 91.0
 x1*exp(1.0*x3) + x2*exp(1.0*x4) - 103.5]


In [30]:
F_0 = lambda x0: np.array([F[i].subs(dict(zip(x, x0))) for i in range(len(F))], dtype = 'float')

In [31]:
def F_Derivative(x0):
    F_der = np.array([[diff(F[i], x[j]) for j in range(len(x))] for i in range(len(F))])
    return np.array([[F_der[i][j].subs(dict(zip(x, x0))) for j in range(len(F_der[i]))] for i in range(len(F_der))], dtype = 'float')

In [32]:
def DividedDifferenceOfOperator(x1, x2):
    arr = []
    for i in range(len(F)):
        subarr = []
        for j in range(len(x1)):
            params1 = [*x1[0:j+1], *x2[j+1:len(x2)+1]]
            params2 = [*x1[0:j], *x2[j:len(x2)+1]]
            result = (F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))) / (x1[j] - x2[j])
            subarr.append(result)
        
        arr.append(subarr)
    
    return np.array(arr, dtype = 'float')

In [33]:
eps = 10 ** -7

In [34]:
def PotraLeastSquaresMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)
    
    I = 0

    A  = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
    operatorsMultiplication = np.dot(A.transpose(), A)
    multiplicationOfOperatorAndFunction = np.dot(A.transpose(), F_0(x0))

    x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I += 1

    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1

        A  = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
        operatorsMultiplication = np.dot(A.transpose(), A)
        multiplicationOfOperatorAndFunction = np.dot(A.transpose(), F_0(x0))

        x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)

        if (max(np.abs(x1 - x0)) < eps and max(np.abs(multiplicationOfOperatorAndFunction)) < eps):
            break
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [35]:
def GaussNewtonLeastSquaresMethod(x0, x, F):
    I = 0

    operatorsMultiplication = np.dot(F_Derivative(x0).transpose(), F_Derivative(x0))

    multiplicationOfOperatorAndFunction = np.dot(F_Derivative(x0).transpose(), F_0(x0))

    x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)
    
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I +=1
    
    while True:
        x0 = x1

        operatorsMultiplication = np.dot(F_Derivative(x0).transpose(), F_Derivative(x0))
        multiplicationOfOperatorAndFunction = np.dot(F_Derivative(x0).transpose(), F_0(x0))

        x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)

        if (max(np.abs(x1 - x0)) < eps and max(np.abs(multiplicationOfOperatorAndFunction)) < eps):
            break
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}\n\n".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [37]:
def LeastSquareMethod(x0):
    print('\n\n')
    print('Norm: {0}'.format(max(F_0(x0))))
    mult = [f * f for f in F_0(x0)]
    return 0.5 * sum(mult)

In [38]:
# approx_values = Example(x0, x, F)
# print(approx_values)

In [39]:
print("Метод Гаусса - Ньютона (найменших квадратів)\n\n")
approx_values = GaussNewtonLeastSquaresMethod(x0, x, F)

LeastSquareMethod(approx_values)

Метод Гаусса - Ньютона (найменших квадратів)


1 Ітерація: Значення F = [ 1.1161869   0.68212736  0.06649451  0.12471195  0.02071833 -0.74056923
 -0.27104889] у точці x = [29.36803638 44.75667557  0.78705813 -0.14515267]
2 Ітерація: Значення F = [ 0.08457395  0.0368298  -0.22411305  0.14034314  0.26767026 -0.37738342
  0.03378816] у точці x = [30.6941881  43.44615504  0.75898878 -0.13479258]


3 Ітерація: Значення F = [ 0.04321262  0.00822918 -0.23907539  0.14057643  0.28556592 -0.33813824
  0.09971019] у точці x = [30.71494964 43.42562679  0.75932614 -0.13433485]


4 Ітерація: Значення F = [ 0.04322974  0.00820126 -0.23910795  0.14056676  0.28558641 -0.33811142
  0.09967287] у точці x = [30.71708794 43.42347882  0.75929681 -0.13435603]


5 Ітерація: Значення F = [ 0.04322785  0.00820263 -0.23910602  0.14056734  0.28558501 -0.33811346
  0.09967444] у точці x = [30.71694974 43.4236176   0.75929868 -0.13435464]


6 Ітерація: Значення F = [ 0.04322797  0.00820254 -0.23910615  0.1405673   

0.1423406509263546

In [40]:
print("Метод Потра (найменших квадратів)\n\n")
approx_values = PotraLeastSquaresMethod(x0, x, F)

LeastSquareMethod(approx_values)

Метод Потра (найменших квадратів)


1 Ітерація: Значення F = [ 1.11618698  0.68212741  0.06649453  0.12471195  0.02071829 -0.74056933
 -0.2710491 ] у точці x = [29.3680369  44.75667505  0.78705812 -0.14515268]
2 Ітерація: Значення F = [ 0.08778746  0.03859556 -0.22364648  0.14066064  0.27048311 -0.36723813
  0.05927695] у точці x = [30.62283094 43.5178297   0.76046707 -0.13430606]
3 Ітерація: Значення F = [ 0.04314639  0.00840102 -0.23888953  0.1406203   0.28541932 -0.33835846
  0.09978182] у точці x = [30.70329199 43.43732831  0.75948429 -0.13421967]
4 Ітерація: Значення F = [ 0.04323958  0.00819416 -0.23911794  0.14056368  0.28559333 -0.33810171
  0.09966262] у точці x = [30.71781223 43.42275145  0.75928698 -0.13436331]
5 Ітерація: Значення F = [ 0.04322719  0.00820309 -0.23910537  0.14056754  0.28558453 -0.33811417
  0.09967495] у точці x = [30.71690259 43.42366495  0.75929932 -0.13435416]
6 Ітерація: Значення F = [ 0.04322801  0.00820251 -0.23910619  0.14056729  0.28558513 -0.33811

0.14234065092635487